<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/Assignment-3/Attention(1-4)/RNN_attention_sweep_q_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K
#from tensorflow.python.keras.layers import Attention as AttentionLayer
import warnings
warnings.filterwarnings('ignore')


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.python import keras


from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import Input,Dense,LSTM,GRU,RNN,SimpleRNN,Softmax,Dropout,Concatenate
from tensorflow.python.keras.layers import TimeDistributed
from tensorflow.python.keras.callbacks import Callback

from tensorflow.python.keras import Model
#from attention import AttentionLayer

from math import log
import math
class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            #de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step,decoder_out_seq, [fake_state_e]
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step,e_outputs, [fake_state_c]
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]


<h2>1.Get the train, val and test Data</h2>

In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xvf '/content/dakshina_dataset_v1.0.tar'


--2022-05-08 14:19:17--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   150MB/s    in 13s     

2022-05-08 14:19:30 (146 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]

dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
d

In [ ]:
train = '/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv'
val =   '/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv'
test =  '/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv'

<h2>2 Preprocessing</h2>

In [ ]:


#this functions extracts the initial data forom the training set provide
# it extracts ::::::
#num_encoder_tokens ,num_decoder_tokens ,
#input_token ,target_token ,reverse_input_token ,
#reverse_target_token,max_encoder_seq_length,max_decoder_seq_length
#
#the above are used further in different functions
def training_extract(path):
    input = []
    target = []
    input_chars = set()
    target_chars = set()
    with open(path, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
    for line in lines[: len(lines) - 1]:
      target_text,input_text, attestation = line.split("\t")
      # We use "tab" as the "start sequence" character
      # for the targets, and "\n" as "end sequence" character.
      input.append(input_text)
      target.append("\t" + target_text + "\n")
      for char in input_text:
        if char not in input_chars:
          input_chars.add(char)
      for char in target_text:
        if char not in target_chars:
          target_chars.add(char)
    target_chars.add("\t")
    target_chars.add("\n")
    input_characters = sorted(list(input_chars))
    target_characters = sorted(list(target_chars))
    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)
    max_encoder_seq_length = max([len(txt) for txt in input])
    max_decoder_seq_length = max([len(txt) for txt in target])
    input_token = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token = dict([(char, i) for i, char in enumerate(target_characters)])

    reverse_input_token = dict((i, char) for char, i in input_token.items())
    reverse_target_token = dict((i, char) for char, i in target_token.items())

    return num_encoder_tokens ,num_decoder_tokens ,input_token ,target_token ,reverse_input_token ,reverse_target_token,max_encoder_seq_length,max_decoder_seq_length


#from here we extract based on the path provided
#the following are extracted 
#input and output (training or test or validation)
#encoder_input_data
#decoder_input_data
#decoder_target_data
def extract_data(path,max_encoder_seq_length,max_decoder_seq_length, num_decoder_tokens):
  input = []
  target = []
  
  with open(path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[: len(lines) - 1]:
    target_text,input_text, attestation = line.split("\t")
    input.append(input_text)
    target.append("\t" + target_text + "\n")
    
    
  encoder_input_data = np.zeros((len(input), max_encoder_seq_length), dtype="float32")
  decoder_input_data = np.zeros((len(input), max_decoder_seq_length), dtype="float32")
  decoder_target_data = np.zeros((len(input),max_decoder_seq_length, num_decoder_tokens), dtype="float32")

  
  for i, (input_text, target_text) in enumerate(zip(input, target)):
    for t, char in enumerate(input_text):
      encoder_input_data[i][t] = input_token[char]
    for t, char in enumerate(target_text):
      decoder_input_data[i][t] = target_token[char]
      if t > 0:
        decoder_target_data[i, t - 1, target_token[char]] = 1.0
  
  return (input ,target ,encoder_input_data,decoder_input_data,decoder_target_data)



<h4>2.1 extract features from train data</h4>

In [ ]:
d=training_extract(train)
print(d)
num_encoder_tokens =d[0]
num_decoder_tokens=d[1]
input_token=d[2]
target_token=d[3]
reverse_input_token=d[4]
reverse_target_token=d[5]
max_encoder_seq_length=d[6]
max_decoder_seq_length=d[7] 

(26, 65, {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}, {'\t': 0, '\n': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'अ': 5, 'आ': 6, 'इ': 7, 'ई': 8, 'उ': 9, 'ऊ': 10, 'ऋ': 11, 'ए': 12, 'ऐ': 13, 'ऑ': 14, 'ओ': 15, 'औ': 16, 'क': 17, 'ख': 18, 'ग': 19, 'घ': 20, 'ङ': 21, 'च': 22, 'छ': 23, 'ज': 24, 'झ': 25, 'ञ': 26, 'ट': 27, 'ठ': 28, 'ड': 29, 'ढ': 30, 'ण': 31, 'त': 32, 'थ': 33, 'द': 34, 'ध': 35, 'न': 36, 'प': 37, 'फ': 38, 'ब': 39, 'भ': 40, 'म': 41, 'य': 42, 'र': 43, 'ल': 44, 'व': 45, 'श': 46, 'ष': 47, 'स': 48, 'ह': 49, '़': 50, 'ा': 51, 'ि': 52, 'ी': 53, 'ु': 54, 'ू': 55, 'ृ': 56, 'ॅ': 57, 'े': 58, 'ै': 59, 'ॉ': 60, 'ो': 61, 'ौ': 62, '्': 63, 'ॐ': 64}, {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v

<h4>2.2 Vectorize the train,val,test data</h4>

In [ ]:
en_ip_tr_text ,de_ip_tr_text , en_ip_tr_data , de_ip_tr_data , de_op_tr_data  = extract_data(train,max_encoder_seq_length,max_decoder_seq_length, num_decoder_tokens)
en_ip_val_text ,de_ip_val_text , en_ip_val_data , de_ip_val_data , de_op_val_data  =extract_data(val,max_encoder_seq_length,max_decoder_seq_length, num_decoder_tokens)
en_ip_ts_text ,de_ip_ts_text , en_ip_ts_data , de_ip_ts_data , de_op_ts_data  = extract_data(test,max_encoder_seq_length,max_decoder_seq_length, num_decoder_tokens)

print("Train data shape : ",en_ip_tr_data.shape)
print("Validation data shape : ",en_ip_val_data.shape)
print("Test data shape : ",en_ip_ts_data.shape)








    


Train data shape :  (44204, 20)
Validation data shape :  (4358, 20)
Test data shape :  (4502, 20)


Utlity fuctions and Word Accuracy Callback

In [ ]:

def beam_search_decoder(data, k):
    decodedWords = [[list(), 0.0]]
    # walk over each step in sequence
    for word in data:
      candidates = list()
      # expand each current candidate
      for sequence in decodedWords:
        seq, score = sequence
        for j in range(len(word)):
          candidate = [seq + [j], score - log(word[j])]
          candidates.append(candidate)
      # order all candidates by score
      ordered = sorted(candidates, key=lambda a:a[1])
      # select k best
      decodedWords = ordered[:k]
    return decodedWords
  
#translating the word back based on the number seqyence given
def translate(word):
  sentence = [] 
  for i in word:
    char = reverse_target_token[i]
    sentence.append(char)
  return "".join(sentence)





In [ ]:

class WordAccuracyCallback(keras.callbacks.Callback):
  def __init__(self,beam_size):
    self.beam_size=beam_size
  def on_epoch_end(self, epoch, logs=None):
    pred=self.model.predict([en_ip_val_data,de_ip_val_data])
    count=0
    for i in range(pred.shape[0]):
      pSequences=beam_search_decoder(pred[i],self.beam_size)
      for j in range(self.beam_size):
        if "\t"+translate(pSequences[j][0][:len(de_ip_val_text[i])-1])==de_ip_val_text[i]:
          count=count+1
          break
    factor = 10.0 ** 4
    logs["WordAccuracy"]=math.trunc((count/pred.shape[0])*factor)/factor
    print("- wordAccuracy:",logs["WordAccuracy"])

<h2>3.Model Class</h2>

In [ ]:
class RNN_Model:

  def __init__(self,embed_size ,no_of_encoder_layers,no_of_decoder_layers,latent_dimension,dropout,recurrent_dropout,cell_type,beam_size):

    self.embed_size = embed_size
    self.no_of_encoder_layers = no_of_encoder_layers
    self.no_of_decoder_layers = no_of_decoder_layers

    self.latent_dimension = latent_dimension
    self.dropout = dropout
    self.recurrent_dropout=recurrent_dropout
    self.cell_type=cell_type
    self.model = None

    #used to store all the encoder layers and dec layers..
    self.input_layers = None
    self.output_layers = None

    #the inference model..
    self.encoder_model = None
    self.decoder_model = None

    #for beam search and greedysearch
    self.beam_size=beam_size

  def BUILD_FIT_MODEL(self, en_ip_tr_data , de_ip_tr_data , de_op_tr_data,epochs ,batch_size,max_encoder_seq_length,num_encoder_tokens,max_decoder_seq_length,num_decoder_tokens):

    #Define the encoder Model..
    encoder_inputs = Input(shape=(max_encoder_seq_length,))
    embd_enc_ip = Embedding(input_dim = num_encoder_tokens,output_dim = self.embed_size ,
                            input_length=max_encoder_seq_length ,name='enc_embd_layer')(encoder_inputs)

    outputs = embd_enc_ip
    encoder_states = []
    self.input_layers = []

    for j in range(self.no_of_encoder_layers):

      if self.cell_type == 'LSTM':
        self.input_layers.append(
            LSTM(self.latent_dimension,return_state=True, return_sequences=True ,recurrent_dropout=self.recurrent_dropout,dropout=self.dropout))
        

      elif self.cell_type == 'GRU':
        self.input_layers.append(
            GRU(self.latent_dimension,return_state=True, return_sequences=True ,recurrent_dropout=self.recurrent_dropout,dropout=self.dropout))

      elif self.cell_type == 'RNN':
        self.input_layers.append(
            SimpleRNN(self.latent_dimension,return_state=True,
                      return_sequences=True ,recurrent_dropout=self.recurrent_dropout,dropout=self.dropout))
        
      all_op = self.input_layers[-1](outputs)
      outputs,encoder_state = all_op[0],all_op[1:]
      encoder_states += [encoder_state]
  
    encoder_outputs=outputs

    #Define The Decoder Model:
    decoder_inputs = Input(shape=(max_decoder_seq_length,))
    embd_dec_ip = Embedding(input_dim = num_decoder_tokens,output_dim = self.embed_size,
                            input_length=max_decoder_seq_length ,name='dec_embd_layer')(decoder_inputs)

    outputs = embd_dec_ip
    self.output_layers = []

    for j in range(self.no_of_decoder_layers):
      if self.cell_type == 'LSTM':
        self.output_layers.append(LSTM(self.latent_dimension,
                                       return_sequences=True, return_state=True ,recurrent_dropout=self.recurrent_dropout,dropout=self.dropout))
      
      elif self.cell_type == 'GRU':
        self.output_layers.append(GRU(self.latent_dimension,
                                       return_sequences=True, return_state=True ,recurrent_dropout=self.recurrent_dropout,dropout=self.dropout))
      
      elif self.cell_type == 'RNN':
        self.output_layers.append(SimpleRNN(self.latent_dimension,
                                       return_sequences=True, return_state=True ,recurrent_dropout=self.recurrent_dropout,dropout=self.dropout))
      
      all_op = self.output_layers[-1](outputs, initial_state = encoder_states[j])
      outputs, dstates = all_op[0],all_op[1:]

    decoder_outputs=outputs

    attn_layer = AttentionLayer(name='attention_layer')
    attn_out, attn_states = attn_layer([encoder_outputs,decoder_outputs])

    decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])
    dense = Dense(num_decoder_tokens, activation='softmax', name='dense_layer')

    dense_time = TimeDistributed(dense, name='time_distributed_layer')
    decoder_pred = dense_time(decoder_concat_input)

    self.model = keras.Model([encoder_inputs, decoder_inputs], decoder_pred)
    self.model.compile(optimizer='adam',loss='categorical_crossentropy' , metrics=['accuracy'])
    self.model.fit(
        [en_ip_tr_data, de_ip_tr_data],
        de_op_tr_data,
        batch_size=batch_size,
        epochs=epochs,
        shuffle=True,
        callbacks = [WordAccuracyCallback(self.beam_size),WandbCallback(monitor='WordAccuracy',
                                                    save_model = True)],verbose=1)




<h2>4. Hyperparameter Tuning Using Wandb</h2>

In [ ]:
!pip install wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 7.9 MB/s 
     |████████████████████████████████| 144 kB 41.8 MB/s 
     |████████████████████████████████| 181 kB 54.7 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=006b3c5e8e9417af53b1234a07106ceeeb1f83f04df2b2a5fcb88870dffb6340
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
sweep_config = {
  'name': 'RNN',
  'method': 'bayes',
  'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
  'parameters': {
        'embed_size':{
            'values':[32,64,128]
        },
        'encoder_layers':{
            'values':[1,2,3]
        },
        'decoder_layers':{
            'values':[1,2,3]
        },
        'hidden_layer_size':{
            'values':[64,128,256]
        },
        'cell_type':{
            'values':['GRU', 'LSTM','RNN']
        },
        'dropout':{
            'values':[0.0,0.3]
        },
        'recurrent_dropout':{
            'values':[0.0]
        },
        'beam_size':{
            'values':[1,3]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='Attention')

Create sweep with ID: 83o9pzql
Sweep URL: https://wandb.ai/pratap49/Attention/sweeps/83o9pzql


In [ ]:
def train():
  run = wandb.init()
  configuration=run.config
  if configuration.encoder_layers == configuration.decoder_layers:

    parent = RNN_Model(embed_size = configuration.embed_size,
                       no_of_encoder_layers = configuration.encoder_layers,
                       no_of_decoder_layers = configuration.decoder_layers,
                       latent_dimension = configuration.hidden_layer_size,
                       dropout = configuration.dropout,
                       recurrent_dropout = configuration.recurrent_dropout,
                       cell_type = configuration.cell_type,
                       beam_size=configuration.beam_size
                       )
    
    
    parent.BUILD_FIT_MODEL(en_ip_tr_data,
               de_ip_tr_data,
               de_op_tr_data,
               10,
                64,
               max_encoder_seq_length ,
               num_encoder_tokens ,
               max_decoder_seq_length ,
                num_decoder_tokens
                )
    
#sweep_id="1ltpguua"
wandb.agent(sweep_id=sweep_id, function=train)

For Testing the Model

In [ ]:
rnn = RNN_Model(128,2,2,256,0,0,"LSTM",3)
rnn.BUILD_FIT_MODEL(en_ip_tr_data,
               de_ip_tr_data,
               de_op_tr_data,
               10,
                64,
               max_encoder_seq_length ,
               num_encoder_tokens ,
               max_decoder_seq_length ,
                num_decoder_tokens
                )